<a href="https://colab.research.google.com/github/SiriN1234/Python_Learning_google_colab/blob/main/chapter_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

출처 : 혼자 공부하는 머신러닝+딥러닝

# 교차 검증과 그리드 서치

- 키워드 : 하이퍼파라미터 (그리드서치 vs 랜덤서치)
- 데이터가 작을 때, 주로 사용
- 하이퍼파라미터
  + max_depth : 3, 정확도가 84%
- 결론
  + 모르면 디폴트만 쓰자
  + 가성비 (시간 대비 성능 보장 안됨)

# 검증 세트

- 테스트 세트 (1회성)
- 훈련 데이터를 훈련 데이터 + 검증 데이터로 재 분할

## 현실

- 테스트 데이터가 별도로 존재하지 않음
- 전체 데이터 = 훈련 (6) : 검증 (2) : 테스트 (2)
  + 테스트 데이터는 모르는 데이터로 생각

# 데이터 불러오기

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42
)

- 검증 세트 만들기

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state = 42
)

In [ ]:
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


# 모델 만든 후 평가

- 과대적합 발생

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증

- 교차 검증의 목적 : 좋은 모델이 만들어진다
  + 좋은 모델 != 성능 좋은 모델
  + 좋은 모델 = 과대적합이 아닌 모델 = 모형의 오차가 적은 모델 = 안정적인 모델
- 교재 245p
  + 모델 평가 1 : 90% (소요시간 : 1시간)
  + 모델 평가 2 : 85%
  + 모델 평가 3 : 80%
- 단점 : 시간이 오래 걸림

# 교차 검증 함수

- 사이킷런에 cross_validate()라는 교차 검증 함수 사용
- 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달, 그 다음 훈련세트 전체를 cross_validate() 함수에 전달

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)

print(scores)

{'fit_time': array([0.02719903, 0.01022577, 0.01074076, 0.01042938, 0.0099504 ]), 'score_time': array([0.00116682, 0.00098372, 0.00113153, 0.00108194, 0.0010345 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 최종점수 평균 구하기
- 검증 폴드의 점수임

In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.8574181117533719


- 훈련 세트 섞은 후, 10-폴드 교차검증

In [ ]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝을 꼭 하고싶다면

- 랜덤 서치를 사용하자
- 자동으로 잡아주는 라이브러리들이 등장하기 시작함
  + hyperopt

## 그리드 서치

- 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함
- 별도로 cross_validate() 함수를 호출할 필요 없음

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- 그리드 서치는 훈련이 끝나면 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자도응로 다시 모델을 훈련 함
- 이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있음
- 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음

In [ ]:
dt = gs.best_estimator_
print(dt)
print(dt.score(train_input, train_target))
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
0.9615162593804117
{'min_impurity_decrease': 0.0001}


- 교차 검증으로 얻은 점수 출력

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


## 랜덤 서치

- 252p
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있도록 확률 분포 객체를 전달
- scipy는 파이썬의 핵심 과학 라이브러리 중 하나. 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리.
- uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음
- 이를 '균등 군포에서 샘플링한다'라고 말 함
- randint는 정수를 뽑고 uniform은 실수를 뽑음

In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([2, 7, 0, 3, 9, 2, 3, 7, 4, 4])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96,  93, 105,  93, 105,  97,  87, 105, 113, 106]))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# p.254

params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001), # 0.0001에서 0.001 사이의 실숫값을 샘플링
    'max_depth' : randint(20, 50), # 20에서 50 사이의 정수를 샘플링
    'min_samples_split' : randint(2, 25), # 2에서 25 사이의 정수를 샘플링
    'min_samples_leaf' : randint(1, 25), # 1에서 25 사이의 정수를 샘플링
}

# params에 정의된 매개변수 범위에서 총 100번(n_iter 매개변수)을 샘플링하여 교차 검증을 수행하고, 최적의 매개변수 조합을 찾음
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter = 100, n_jobs = -1, random_state = 42)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fed0d11b210>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fed0d11b2d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fed0d14fe90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fed0d0f33d0>},
                   random_state=42)

- 최적의 매개변수 조합 출력

In [ ]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

# 마무리

## 키워드로 끝내는 핵심 포인트

- **검증 세트**는 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트입니다.
- **교차 검증**은 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련합니다. 교차 검증은 이런 식으로 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법입니다.
- **그리드 서치**는 하이퍼파라미터 탐색을 자동화해 주는 도구입니다. 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택합니다. 마지막으로 이 매개변수 조합으로 최종 모델을 훈련합니다.
- *랜덤 서치**는 연속된 매개변수 값을 탐색할 때 유용합니다. 탐색할 값을 직접 나열하는 것이 아니고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달합니다. 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있습니다.

## 핵심 패키지와 함수

### scikit-learn

- **cross_validate()**는 교차 검증을 수행하는 함수힙니다
  + 첫 번째 매개변수에 교차 검증을 수행할 무델 객체를 전달합니다. 두 번째와 세 번째 매개변수에 특성과 타깃 데이터를 전달합니다.
  + scoring 매개변수에 검증에 사용할 평가 지표를 지정할 수 있습니다. 기본적으로 분류 모델은 정확도를 의미하는 'accuracy', 회귀 무델은 결정계수를 의미하는 'r2'가 됩니다.
  + cv 매개변수에 교차 검증 폴드 수나 스플리터 객체를 지정할 수 있습니다. 기본값은 5입니다. 회귀할 때는 KFold 클래스를 사용하고 분류일 때는 StratifiedKFold 클래스를 사용하여 5-폴드 교차 검증을 수행합니다
  + n_jobs 매개변수는 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정합니다. 기본값은 1로 하나의 코어를 사용합니다. -1로 지정하면 시스템에 있는 모든 코어를 사용합니다.
  + return_train_score 매개변수를 True로 지정하면 훈련 세트의 점수도 반환합니다. 기본값은 False입니다.

- **GridSearchCV**는 교차 검증으로 하이퍼파라미터 탐색을 수행합니다. 최상의 무델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련합니다.
  + 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달합니다. 두 번째 매개변수에는 탐색할 모델의 매개변수와 값을 전달합니다.
  + scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일합니다.

- **RandomizedSearchCV**는 교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행합니다. 최상의 무델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련합니다.
  + 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달합니다. 두 번째 매개변수에는 탐색할 모델의 매개변수와 확률 분포 객체를 전달합니다.
  + scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일합니다.